# Exploratory Data Analysis of Palladium Spot Price

Data is provided free for non-commercial use by Perth Mint (www.perthmint.com)

## The Task

Extract-Load-Transform (ELT) process of transforming noisy data in CSV files (e.g. inconsistent data types, spurious values, etc.) into cleaned data suitable for increasingly sophisticated analysis.

In [ ]:
# IMPORTS
import locale
import re
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from datetime import datetime
from dateutil.parser import parse

# CONSTANTS
DATE_LINE_RE = re.compile('^"?\d+')
DATE_FORM_RE = re.compile('^"?(\d+)/(\d+)/(\d+)"?\s*(.*)$')

This function ensures pandas will not throw exceptions without a good reason.

In [ ]:
def locale_string_to_float(value: str) -> float:
    """Attempt to convert the argument to a float."""
    
    returnVal = np.nan
    try:
        returnVal = locale.atof(value)
    except ValueError as error:
        pass
    
    return returnVal

A specific problem with the CSV files occurs when dates cross from the 10th to the 11th of some months.

In [ ]:
def clean_data(filename: str, after_millenium: bool) -> None:
    """Create a new CSV file from ISO-correct dates."""
    
    with open(filename, 'r', encoding='utf-8') as infile:
        old_date = datetime(1900, 1, 1)
        millenium_crossed = after_millenium
        with open(filename + '.cleaned.csv', 'w', encoding='utf-8') as outfile:
            for line in infile:
                if DATE_LINE_RE.match(line):
                    line = line.strip()
                    matched = DATE_FORM_RE.match(line)
                    if matched:
                        year_str = matched[3]
                        if not millenium_crossed:
                            if year_str == '00':
                                millenium_crossed = True
                            else:
                                year_str = f"19{year_str}"
                                
                        if millenium_crossed:
                            year_str = f"20{year_str}"
                            
                        maybe_month = matched[2]
                        maybe_day = matched[1]
                        current_date = parse(
                            f"{maybe_day}/{maybe_month}/{year_str}"
                        )
                        if abs((current_date - old_date).days) > 1:
                            current_date = parse(
                                f"{maybe_month}/{maybe_day}/{year_str}"
                            )
                        print(
                            f"{current_date}{matched[4]}",
                            file=outfile
                        )
                        old_date = current_date

A function to remove outliers in a given pandas dataframe. The valid column name is given as well as a floating point threshold factor. All rows with values too far above or below the threshold are returned as a list.

In [ ]:
def abnormal_value_indices(df: pd.DataFrame, column_name: str, threshold_factor: float) -> list:
    """Returns rows in a dataframe exceeding a threshold."""
    
    abnormals_list = []
    previous_value = df[column_name][0]
    for i in range(1, len(df[column_name])):
        current_value = df[column_name][i]
        if (previous_value > current_value * threshold_factor) or (current_value > previous_value * threshold_factor):
            abnormals_list.append(df.index[i])
        else:
            previous_value = current_value
    print(abnormals_list)
    
    return(abnormals_list)

Create CSV files filled with clean (albeit sparse) data based on the original CSV files.

In [ ]:
clean_data(r'Data\part_1.csv', False)

In [ ]:
clean_data(r'Data\part_2.csv', True)

Data is split between the 2 CSV files. There are prices from 1968 to 2015 in one, and from 2016 onwards in the other.

In [ ]:
df_london_fixes_daily_1968_2015 = pd.read_csv(
    r'Data\part_1.csv.cleaned.csv',
    encoding='utf-8',
    header=None,
    names=['Date', 'Au AM', 'Au PM', 'Ag', 'Pt AM', 'Pt PM', 'Pd AM', 'Pd PM'],
    index_col=0, 
    skiprows=[0,1,2,3,4],
    usecols=[0,1,2,3,4,5,6,7],
    parse_dates=True,
    converters={
        1: locale_string_to_float,
        2: locale_string_to_float,
        3: locale_string_to_float,
        4: locale_string_to_float,
        5: locale_string_to_float,
        6: locale_string_to_float,
        7: locale_string_to_float
    }
)

The dataframe deliberately holds NaN values as compaction will be done just before the data is needed.

In [ ]:
df_london_fixes_daily_1968_2015.head()

In [ ]:
df_london_fixes_daily_from_2016_on = pd.read_csv(
    r'Data\part_2.csv.cleaned.csv',
    encoding='utf-8',
    header=None,
    names=['Date', 'Au AM', 'Au PM', 'Ag', 'Pt AM', 'Pt PM', 'Pd AM', 'Pd PM'],
    index_col=0,
    skiprows=[0,1,2,3,4],
    usecols=[0,1,2,3,4,5,6,7],
    parse_dates=True,
    converters={
        1: locale_string_to_float,
        2: locale_string_to_float,
        3: locale_string_to_float,
        4: locale_string_to_float,
        5: locale_string_to_float,
        6: locale_string_to_float,
        7: locale_string_to_float
    }
)

In [ ]:
df_london_fixes_daily_from_2016_on.head()

Ready for basic analysis, we drop NaN entries in specific columns and then drop outliers. Here the outliers are values that are bigger or smaller than their preceding value by some factor.

In [ ]:
data_d1 = {
    'Pd AM': df_london_fixes_daily_1968_2015['Pd AM'],
    'Pd PM': df_london_fixes_daily_1968_2015['Pd PM']
}
df_palladium_1 = pd.DataFrame(data_d1).dropna(axis=0)
df_palladium_1 = df_palladium_1.drop(abnormal_value_indices(df_palladium_1, 'Pd AM', 1.5))
df_palladium_1.describe()

In [ ]:
data_d2 = {
    'Pd AM': df_london_fixes_daily_from_2016_on['Pd AM'],
    'Pd PM': df_london_fixes_daily_from_2016_on['Pd PM']
}
df_palladium_2 = pd.DataFrame(data_d2).dropna(axis=0)
df_palladium_2 = df_palladium_2.drop(abnormal_value_indices(df_palladium_2, 'Pd AM', 1.5))
df_palladium_2.describe()

Combine the compacted dataframes to analyze the data contiguously from 1968 to 2016 onwards.

In [ ]:
df_palladium = df_palladium_1.append(df_palladium_2)
df_palladium.describe()

In [ ]:
fig, ax = plt.subplots()
ax.plot(df_palladium['Pd AM'])
ax.grid()
fig.set_size_inches(15, 5)
plt.title("Historical Daily Palladium Spot Prices (AM)")
plt.xlabel("Date")
plt.ylabel("Price / USD")
plt.show()

### Appendix

Visualizations to see the data transformations.

##### 1. Raw data

In [ ]:
df_london_fixes_daily_1968_2015_raw = pd.read_csv(
    r'Data\part_1.csv', 
    encoding='utf-8',
    header=None,
    names=['Date', 'Au AM', 'Au PM', 'Ag', 'Pt AM', 'Pt PM', 'Pd AM', 'Pd PM'],
    index_col=0, 
    skiprows=[0,1,2,3,4],
    usecols=[0,1,2,3,4,5,6,7],
    parse_dates=True,
    converters={
        1: locale_string_to_float,
        2: locale_string_to_float,
        3: locale_string_to_float,
        4: locale_string_to_float,
        5: locale_string_to_float,
        6: locale_string_to_float,
        7: locale_string_to_float
    }
)
fig, ax = plt.subplots()
ax.plot(df_london_fixes_daily_1968_2015_raw['Pd AM'])
ax.grid()
fig.set_size_inches(15, 5)
plt.title("Palladium Spot Price (AM) - Raw Data (part_1.csv) [1968-2015]")
plt.xlabel("Date")
plt.ylabel("Price / USD")
plt.show()

In [ ]:
df_london_fixes_daily_2016_on_raw = pd.read_csv(
    r'Data\part_2.csv', 
    encoding='utf-8',
    header=None,
    names=['Date', 'Au AM', 'Au PM', 'Ag', 'Pt AM', 'Pt PM', 'Pd AM', 'Pd PM'],
    index_col=0, 
    skiprows=[0,1,2,3,4],
    usecols=[0,1,2,3,4,5,6,7],
    parse_dates=True,
    converters={
        1: locale_string_to_float,
        2: locale_string_to_float,
        3: locale_string_to_float,
        4: locale_string_to_float,
        5: locale_string_to_float,
        6: locale_string_to_float,
        7: locale_string_to_float
    }
)
fig, ax = plt.subplots()
ax.plot(df_london_fixes_daily_2016_on_raw['Pd AM'])
ax.grid()
fig.set_size_inches(15, 5)
plt.title("Palladium Spot Price (AM) - Raw Data (part_2.csv) [2016+]")
plt.xlabel("Date")
plt.ylabel("Price / USD")
plt.show()

##### 2. Initial data cleaning 

In [ ]:
fig, ax = plt.subplots()
ax.plot(df_london_fixes_daily_1968_2015['Pd AM'])
ax.grid()
fig.set_size_inches(15, 5)
plt.title("Palladium Spot Price (AM) - Initial Clean Data (part_1.csv.cleaned.csv) [1968-2015]")
plt.xlabel("Date")
plt.ylabel("Price / USD")
plt.show()

3 extreme values are seen in the plot above: 2005-05-18, 2012-02-29, and 2012-10-04.

In [ ]:
fig, ax = plt.subplots()
ax.plot(df_london_fixes_daily_from_2016_on['Pd AM'])
ax.grid()
fig.set_size_inches(15, 5)
plt.title("Palladium Spot Price (AM) - Initial Clean Data (part_2.csv.cleaned.csv) [2016+]")
plt.xlabel("Date")
plt.ylabel("Price / USD")
plt.show()

No extreme values seen in the plot above. The same data will still be run through the outlier isolation function.

##### 3. Outlier removal

In [ ]:
fig, ax = plt.subplots()
ax.plot(df_palladium_1['Pd AM'])
ax.grid()
fig.set_size_inches(15, 5)
plt.title("Palladium Spot Price (AM) - Outliers Removed [1968-2015]")
plt.xlabel("Date")
plt.ylabel("Price / USD")
plt.show()

In [ ]:
fig, ax = plt.subplots()
ax.plot(df_palladium_2['Pd AM'])
ax.grid()
fig.set_size_inches(15, 5)
plt.title("Palladium Spot Price (AM) - Outliers Removed [2016+]")
plt.xlabel("Date")
plt.ylabel("Price / USD")
plt.show()